# 13. Naive Bayes

Uma rede social não é tão boa se as pessoas não conseguem se conectar. Portanto, a DataSciencester possui um atributo popular que permite que membros enviem mensagens uns aos outros. E enquanto a maioria dos membros são cidadãos responsáveis que somente enviam mensagens de “como você está?”, alguns são canalhas e enviam mensagens de spam sobre esquemas para ficarem ricos, medicamentos sem receita e programas de credenciamento de data science. Seus usuários começaram a reclamar e a Vice-presidente de Mensagem pediu que você usasse data science para descobrir como filtrar essas mensagens de spam.

### Um Filtro de Spam Muito Estúpido

* Imagine um "universo" que consiste em receber uma mensagem escolhida ao acaso entre todas as possíveis.
  * S: a mensagem é spam.
  * V: A mensagem contém a palavra "viagra".
  
  
* O teorema de Bayes (Cap. 06) diz que a probabilidade de uma mensagem ser spam depender da palavra "viagra" é dado por:

   $P(S|V) = [P(V|S)*P(S)] / [P(V|S)*P(S)+P(V|not(S))*P(not(S))]$

* Esse cálculo pode ser visto como a proporção de mensagens com a palavra "viagra" que são spam.
* Se temos uma grande quantidade de mensagens que sabemos que não são spam e outra grande qauntidade que sabemos ser spam, então é possível calcular $P(V|S)$ e $P(V|not(S))$.
* Se presumimos que uma mensagem possui igual probabilidade de ser spam ou não spam ($P(S) = P(not(S)) = 0.5$), então:

   $P(S|V) = P(V|S)/[P(V|S)+P(V|not(S))]$

* Ex.: 50% das mensagens de spam possuem a palavra "viagra". 1% das mensagens não spam possuem. A probabilidade de que qualquer email que tenha a palavra "viagra" seja spam é:

   $0.5/(0.5+0.01) = 98\%$

### Um Filtro de Spam Mais Sofisticado

* Imagine que temos um vocabulário de muitas palavras $w_{1}$, ..., $w_{n}$.
* Usaremos $X_{i}$ para o evento "a mensagem contém a palavra $w_{i}$".
* Imagine também que encontramos um $P(X_{i}|S)$ como estimativa da probabilidade de a mensagem de spam conter a palavra $w_{i}$ e $P(X_{i}|not(S))$ de a mensagem de não spam conter a palavra $w_{i}$.

* A chave para naive bayes é fazer a suposição de que as presenças (ou ausências) de cada palavra são independentes uma das outras, condição para uma mensagem ser spam ou não.
  * Significa que a presença de uma palavra na mensagem não indica a presença de outra. Matematicamente:
  
    $P(X_{1} = x_{1}, ..., X_{n} = x_{n}|S) = P(X_{1} = x_{1}|S) * ... * P(X_{n} = x_{n}|S)$
    
* É uma hipótese extrema.
* Imagine que todo o nosso vocabulário consista apenas das palavras “viagra” e “rolex”, e que metade de todas as mensagens de spam sejam “viagra barato” e que a outra metade seja “rolex autêntico”.
* A estimativa que um spam conhtenha os dois é:

   $P(X_{1} = 1, X_{2} = 1|S) = P(X_{1} = 1|S)*P(X_{2} = 1|S) = 0.5*0.5 = 0.25$
   
* assim, afastamos a teoria de que "viagra" e "rolex" não apareçam juntos.
* O Teorema de Bayes também diz que podemos calcular a probabilidade de uma mensagem ser spam usando a equação:

   $P(S|X = x) = P(X = x|S)/[P(X = x|S)+P(X = x|not(S))]$
   
* A suposição de Naive Bayes permite calcular cada uma das probabilidades multiplicando junto as estimativas de probabilidade de cada uma das palavras.
* Na prática a múltiplicação de muitas probabilidades ao mesmo tempo é evitada para evitar underflow.
* Esse cálculo então é feito de forma mais amigável ao computador usando a seguinte fórmula:

   $exp(log(p_{1})+...+log(p_{n}))$
   
* Não devemos calcular a probabilidade de $P(X_{i}|S)$ como uma fração de mensagens contendo a palavra $w_{i}$.
* Isso, quando  pode fazer com que a probabilidade da mensagem ser spam seja 0, mesmo que a mensagem tenha muitas outras palavras que a indiquem como um spam.
* Para esse calculo utilizamos um valor k para que a probabilidade da palavra tenha um valor mínimo que não prejudique o resultado final. A fórmula final fica assim:

   $P(X_{i}|S)$ = (k + número de spams contendo $w_{i}$ / (2k + número de spams)
   
* O mesmo deve ser feito para $P(X_{i}|S)$.
* Ao adicionar k presumimos que mesmo que não apareça a palavra em nenhuma mensagem de spam, é possível que ela apareça em alguma mensagem de spam desconhecida.
* Ex.: se "dado" ocorre em 0/98 documentos spam e se k = 1, $P("dado"|S)$ é calculado como:

   $1/100 = 0.001$
   
* Isso permite que o classificador atribua alguma probabilidade diferente de 0 para mensagens que contenham a palavra "dado".

### Implementação

* Iniciamos criando uma função bem simples para quebrar mensagens em palavras distintas.
* A segunda função contará as palavras em um conjunto de palavras rotuladas para treino.
  * O retorno será um dicionário no qual as palavras são as chaves e os valores são listas com 2 valores indicando a quantidade de vezes que a palavra foi vista em mensagens spam e em mensagens não spam.

In [3]:
import re
from collections import defaultdict

# Função para quebrar mensagens
def tokenize(message):
    message = message.lower() # Converte tudo para minúsculo
    all_words = re.findall("[a-z0-9']+", message) # Extrai as palavras
    return set(all_words) # Remove duplicatas

# Função para contar palavras
def count_words(training_set):
    counts = defaultdict(lambda: [0,0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1]+=1
    return counts

* O passo seguinte é transformar as contagens em probabilidades.
  * O retorno será uma lista de triplas contendo (palavra, prob spam, prob não spam)
  

* A última parte será usar as probabilidades das palavras para atribuir probabilidades a mensagens:

In [4]:
import math

# Calcula a probabilidade das palavras
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    return [(w,
            (spam+k)/(total_spams+2*k),
            (non_spam+k)/(total_non_spams+2*k))
           for w, (spam, non_spam) in counts.items()]

# Usa as probabilidades das palavras para atribuir probabilidades as mensagens
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    # Itera sobre cada palavra em nosso vocabulário
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        
        # se "word" aparecer na mensagem,
        # adicione a probabilidade log de vê-la
        if word in message_words:
            log_prob_if_spam+=math.log(prob_if_spam)
            log_prob_if_not_spam+=math.log(prob_if_not_spam)
        
        # se "word" não aparecer na mensagem
        # adicione a probabilidade log de não vê-la
        # que é log(1-probabilidade de vê-la)
        else:
            log_prob_if_spam+=math.log(1.0-prob_if_spam)
            log_prob_if_not_spam+=math.log(1.0-prob_if_not_spam)
        
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam+prob_if_not_spam)

* E então podemos juntar tudo isso no classificador:

In [21]:
class NaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
        
    def train(self, training_set):
        
        # Conta mensagens de spam e não spam
        num_spams = len([is_spam
                        for message, is_spam in training_set
                        if is_spam])
        num_non_spams = len(training_set)-num_spams
        
        # roda dados de treinamento pela nossa "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                            num_spams,
                                            num_non_spams,
                                            self.k)
        
    def classify(self, message):
        return spam_probability(self.word_probs, message)

### Testando o Nosso Modelo

* Para o teste do nosso modelo, utilizaremos o conjunto de dados SpamAssassin public corpus, apesar de ser um conjunto antigo.
* Para o teste utilizaremos apenas o assunto de cada email.
* Todas as linhas referentes ao assunto começam com "Subject:"
  * Procuraremos por isso.

In [38]:
# Funções auxiliares
# Cap. 11
random.seed(0)
def split_data(x, prob):
    results = [],[]
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

def accuracy(tp, fp, fn, tn):
    correct = tp+tn
    total = tp+fp+fn+tn
    return correct/total

# Sensibilidade
def recall(tp, fp, fn, tn):
    return tp / (tp + fn)

In [18]:
import glob

path = 'Dados/spam/*/*'

data = []

for fn in glob.glob(path):
    is_spam = "ham" not in fn
    
    with open(fn, 'r', encoding="utf8", errors='ignore') as file:
        for line in file:
            if line.startswith("Subject:"):
                # remove o primeiro "Subject:" e mantém o que sobrou
                subject = re.sub(r"^Subject: ", "", line).strip()
                data.append((subject, is_spam))

In [28]:
import random
from collections import Counter
# Iremos dividir o nosso conjunto em Treino e Teste

train_data, test_data = split_data(data, 0.75)

# Realiza o Treino
classifier = NaiveBayesClassifier()
classifier.train(train_data)

# Verificando como o nosso modelo faz
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

# presuma que spam_probability > 0.5 corresponde à previsão de spam
# e conta as combinações de (is_spam real, is_spam previsto)
counts = Counter((is_spam, spam_probability > 0.5) 
                 for _, is_spam, spam_probability in classified)

In [42]:
# Obtendo os resultados e calculando as métricas
tp = counts.get((True, True))
fp = counts.get((False, True))
fn = counts.get((True, False))
tn = counts.get((False, False))
print("Acurácia: {:.2f}%".format(accuracy(tp, fp, fn, tn)*100))
print("Sensibilidade: {:.2f}%".format(recall(tp, fp, fn, tn)*100))

Acurácia: 90.18%
Sensibilidade: 65.67%


In [46]:
# Vamos dar uma olhada nos mais mal classificados
classified.sort(key=lambda row: row[2])

# as maiores probabilidades de spam previstos entre os não-spams
spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]

# as menores probabilidades de spam previstos entre os spams
hammiest_spams = list(filter(lambda row: row[1], classified))[:5]

In [51]:
print(spammiest_hams)

[('The MIME information you requested (last changed 3154 Feb 14)', False, 0.9977012929188156), ('Save up to 70% on international calls!', False, 0.998256216404264), ('Four free e-mailers reviewed, Get the gear you need to burn DVDs', False, 0.9993376854790142), ('=?iso-8859-1?Q?Matrox_Parhelia=99_now_available?=', False, 0.999692131524301), ('"I meditated in a cave for 12 years and now I\'m here to tell you', False, 0.9999830867163033)]


In [52]:
print(hammiest_spams)

[('I was so scared... my very first DP', True, 6.082339995944272e-05), ('Re: girls', True, 0.0008412798514905131), ('.Message report from your contact page....//ytu855 rkq', True, 0.0021595543693716636), ('Looking for property in SPAIN?', True, 0.002842605425644866), ('[scoop] ....It is not my fault. .- vwiid', True, 0.0046969498498913775)]


In [48]:
# Podemos ver quais as palavras que possuem mais jeito de spam
def p_spam_given_word(word_prob):
    """usa o teorema de bayes para computar p(spam | message contains word)"""
    # word_prob é uma das triplas produzidas por word_probabilities
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]

In [49]:
spammiest_words

[('sale', 0.031081081081081083, 0.0002294630564479119),
 ('money', 0.03648648648648649, 0.0002294630564479119),
 ('systemworks', 0.03648648648648649, 0.0002294630564479119),
 ('adv', 0.03918918918918919, 0.0002294630564479119),
 ('rates', 0.041891891891891894, 0.0002294630564479119)]

In [50]:
hammiest_words

[('spambayes', 0.0013513513513513514, 0.04749885268471776),
 ('users', 0.0013513513513513514, 0.040614960991280404),
 ('was', 0.0013513513513513514, 0.03832033042680129),
 ('razor', 0.0013513513513513514, 0.0346489215236347),
 ('sadev', 0.0013513513513513514, 0.03051858650757228)]

* Como poderíamos obter melhores resultados?
  * A maneira óbvia seria obter mais dados para treino.
  

* Existem várias maneiras de melhorar o modelo.
* Algumas possibilidades são:
  * Olhe o conteúdo da mensagem todo.
  * O classificador implementado leva em consideração cada palavra que aparece no conjunto de treinamento, até mesmo as palavras que só aparecem uma vez. Modifique o classificador para aceitar um limite opcional min_count e ignore os símbolos que não aparecem tantas vezes.
  * O tokenizer não tem percepção de palavras similares (por exemplo, “cheap” e “cheapest”). Modifique o classificador para ter uma função stemmer que converte palavras para as classes equivalentes de palavras.
    * Criar uma boa função stemmer é difícil. Porter Stemmer é geralmente a mais usada.
  * Mesmo que todas as nossas características sejam “mensagens contendo a palavra $w_{i}$ ́ ”, não há motivo para tal. Em nossa implementação, nós pudemos acrescentar características extras como “mensagem contendo um número” criando tokens fictícios como contains:number e modificando o tokenizer para emiti-los quando necessário.